# Import the necessary libraries and packages

In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import json
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Preparing to scrape data - download BeautifulSoup

In [2]:
from bs4 import BeautifulSoup
print ("libraries downloaded")

libraries downloaded


In [11]:
toronto_data = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
scraper = BeautifulSoup(toronto_data,'lxml')


In [4]:
tables = scraper.find('table',{'class':'wikitable sortable'})

In [5]:
df = pd.read_html(str(tables))
df = pd.DataFrame(df[0])

* We ignore the entries in the Borough column that are "Not assigned". If more than one neighbourhood share the same postal code then the neighbourhoods are combined in one row and separated by a comma. I did not need to write code for it since the table scraped from the wikipedia page above had already been formatted. 

In [6]:
df = df[df.Borough != "Not assigned"]
df = df.reset_index(drop = True)
print('The dataframe has {} columns and {} rows'.format(df.shape[1],df.shape[0]))
df

The dataframe has 3 columns and 103 rows


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


# Assigning coordinates to neighbourhoods based on Postal Codes

In [7]:
import geocoder

In [8]:
def get_latlng(neighbourhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Canada'.format(neighbourhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords
# Call the function to get the coordinates, store in a new list using list comprehension
coordinates = [get_latlng(neighbourhood) for neighbourhood in df["Postal Code"].tolist()]

In [9]:
df_coordinates = pd.DataFrame(coordinates,columns = ['Latitude','Longitude'])
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']
print('The dataframe has {} rows and {} columns'.format(df.shape[0],df.shape[1]))
df.head()

The dataframe has 103 rows and 5 columns


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.66253,-79.39188


In [10]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 103 neighborhoods.
